In [ ]:
#@title **MANGA WATCH**
#@markdown **A Web Scrapping App to Read Manga**
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual, Layout
from ipywidgets import Button, HBox, VBox

from IPython.display import clear_output
from IPython.display import display, HTML, Image

from bs4 import BeautifulSoup
import requests
import re
import functools
import time

#Title of Project with HTML widget
Title = "<html><head><style>.code-style {font-size: 30px;line-height: 30px;color: red;} </style></head><code class=" +'"code-style"> MANGA WATCH' + "</code></html>"
Title = widgets.HTML(value= Title)

#Widget for Search text
Search = widgets.Text(
    value='',
    placeholder='Enter Manga Name',
    disabled=False
)

#widget for Search Button
Search_Button = widgets.Button(
    description='Search',
    disabled=False,
)

#Widget positioning Title with Search boxes
Hbox_1=HBox([Search,Search_Button])
Vbox_1=[Title,Hbox_1]

# Global Variables for Search 
Search_List = []                  #list to capture manga search list
Cur_Search_Page = 1               #variable to track current search page
Tot_Search_Page = 1               #variable to track Total search page


# Global Variables for Chapter in Manga
Chap_Link_List = []               #List to keep links of manga chapters
Chap_Title_List = []              #List to keep titles of Manga Chapters
Cur_Man = 0                       #variable to track current manga
Cur_Chap = 1                      #variable to track current manga current chapter
Tot_Chap = 1                      #variable to track current manga total chapters

# Global Variables for Image Display
Image_list = []                   #list to keep Imag source Url
Tot_Man_Page = 0                  #variable to track current manga chapter page
Cur_Man_Page = 0                  #variable to track current manga chapter total pages


#Function to get search List from Search string
def Search_List_fun(d):
  #Sleaap for 1 second before reading search text
  time.sleep(1)

  #listout global variables
  global Search_List
  global Cur_Search_Page
  global Tot_Search_Page

  #intialize global variables
  Search_List = []
  Sl_No = 1

  #Proceed only if Search text is not empty
  if (Search.value!=''):

    #Get Search url from Current search text and search page
    search_url = 'https://mangaclash.com/page/'+str(Cur_Search_Page)+ '/?s=' + Search.value +'&post_type=wp-manga'

    #get web page data from requests 
    search_page = requests.get(search_url)

    #Parse web page and get lxml data
    search_soup = BeautifulSoup(search_page.text, "lxml")

    Temp_list = [] #temporaty list

    #Convert parsed webpage data to string and go thrigh line by line
    for link in str(search_soup).split('\n'):

        #To get Manga titles and link by web scrapping
        if ('<h3 class="h4"' in link ):

              #add Sl_No. Button(disabled) Widget
              Temp_list.append(widgets.Button(description=str(Sl_No), disabled=True,layout = widgets.Layout(min_width='300px', height='40px')))

              #add Manag Title Button(enables) widget
              Temp_list.append(widgets.Button(description=re.search(r'/">(.*?)</a>', link).group(1), disabled=False,layout = widgets.Layout(min_width='800px', height='40px')))

              #Add manga Title string
              Temp_list.append(re.search(r'/">(.*?)</a>', link).group(1))

              #Add Manga Link string
              Temp_list.append(re.search(r'href="(.*?)"', link).group(1))
          
        #T0 Get Latest chapter by web scrapping
        if ('<span class="font-meta chapter"' in link):

              #Add Latest chapter Button(disabled) widget 
              Temp_list.insert(2,widgets.Button(description=re.search(r'/">(.*?)</a', link).group(1), disabled=True,layout = widgets.Layout(min_width='300px', height='40px')))

              #Append Temp list to Global Search_list
              Search_List.append(Temp_list)

              #Increment SL No.
              Sl_No+=1

              #Reinitialise Temp List
              Temp_list=[]

        #To get Total Search Page by Web Scrapping      
        if ('<span class="pages"' in link and Cur_Search_Page == 1):

              #Update global Tot_Search_Page
              Tot_Search_Page = int(re.search(r' of (.*?)</s', str(link)).group(1))          

    #Call function to display Search List                  
    Final_Disp(1,ind=1)    #Argument contains Activity, Index


#Function to get Chapter List from Selected Manga
def Get_Chap_List(b,ind=0):

    #listout global variables
    global Search_List
    global Chap_Link_List
    global Chap_Title_List
    global Cur_Chap
    global Tot_Chap
    global Cur_Man

    #intialize global variables
    Chap_Link_List=[]
    Chap_Title_List=[]

    #Update Current Manga
    Cur_Man = ind

    #Get Manga Chapter List url from Search List index data is passed by On click Manga Title widget
    #get web page data from requests
    Chap_page = requests.get(Search_List[ind][4])

    #Parse web page and get lxml data
    Chap_soup = BeautifulSoup(Chap_page.text, "lxml")
    
    #get all li headers from lxml
    for link in Chap_soup.find_all("li"):

        #Get all lines which has the substring in it
        if ('class="wp-manga-chapter "' in str(link) ):

          #Add Chapters to Chapter list 
          Chap_Title_List.append(re.search(r'\n(.*?)</a>', str(link)).group(1))

          #Add Chapter Urls to Chapter Link List
          Chap_Link_List.append(re.search(r'<a href="(.*?)"', str(link)).group(1))

    #Update Total number of Chapter     
    Tot_Chap = len(Chap_Title_List)

    #update latest Chapter has current chapter
    Cur_Chap = Tot_Chap

    #Display Current chapter Images
    Disp_Image(0,ind=Cur_Chap)


#Function to get Image Source URL from Chapter Url
def Disp_Image(b,ind):
  #listout global variables
  global Tot_Man_Page
  global Cur_Man_Page
  global Image_list
  global Cur_Chap

  #intialize required global variables
  Image_list = []

  #Update Current Chapter
  Cur_Chap = ind

  #Get Image Page Url from Current Chapter Url
  #get web page data from requests
  Image_page = requests.get(Chap_Link_List[Tot_Chap - Cur_Chap])   #index is (Tot_Chap - Cur Chap) sinse latest chapter is of index 0

  #Parse web page and get lxml data
  Image_soup = BeautifulSoup(Image_page.text, "lxml")

  #get all img headers from lxml
  for link in Image_soup.find_all("img"):

        #Get all lines which has the substring in it
        if ('class="wp-manga-chapter-img img-responsive lazyload effect-fade"' in str(link) ):

          #Add Image source Url to Image List
          Image_list.append(re.search(r'\n(.*?)" id="', str(link)).group(1).strip())

  #Update Total Image count in a chapter         
  Tot_Man_Page = len(Image_list)

  #update 1st Image page has current Image page
  Cur_Man_Page = 1

  ##Call function to display latest chapter , 1st image of current manga from Image List
  Final_Disp(1, ind=2)


#Function to Navigate pages in Activity 
def Nav_Page(b,tag,ind):

   #listout global variables
  global Cur_Man_Page
  global Tot_Man_Page
  global Cur_Search_Page
  global Tot_Search_Page

  #index 1 is for Search list display
  #Update Current Search page from page navigation buttons pressed
  if(ind==1):

    #For First Page
    if(tag==0):
      Cur_Search_Page = 1

    #For Previous Page
    elif(tag==1):
      Cur_Search_Page-=1

    #for Next Page  
    elif(tag==2):
      Cur_Search_Page+=1

    #for Last Page
    elif(tag==3):
      Cur_Search_Page=Tot_Search_Page

    #Call Search List funtion to get Manga list from Current Search Page  
    Search_List_fun(0)
  
  #index 2 is for Image Page display
  #Update Current Manga Image page from page navigation buttons pressed
  if (ind==2):

    #For First Image
    if(tag==0):
      Cur_Man_Page = 1

    #For Previous Image  
    elif(tag==1):
      Cur_Man_Page-=1

    #For Next Image  
    elif(tag==2):
      Cur_Man_Page+=1

    #For Last Image  
    elif(tag==3):
      Cur_Man_Page=Tot_Man_Page

    #Call Funtion to Display Current Image Page
    Final_Disp(1,ind=2)
  

#Function to Navigate Chapter Selection Dropdown
def On_Change_Chapter(change):
    #Get Value of Chapter Change
    if change['type'] == 'change' and change['name'] == 'value':

        #Get index of Chapter Selected
        index =Chap_Title_List.index(change['new'])

        #If Chapter Selected is Diffrent to Current Chapter Proceed to Display 1st Image of New Chapter
        if(index != Tot_Chap - Cur_Chap):
          Disp_Image(0,Tot_Chap - index)

#Function to Navigate Pages Selection Dropdown
def On_Change_Page(change,ind):

    #listout global variables
    global Cur_Man_Page
    global Tot_Man_Page
    global Cur_Search_Page
    global Tot_Search_Page

    #Get Value of Page Change
    if change['type'] == 'change' and change['name'] == 'value':

      #index 1 is for Search Manga Pages
      if(ind==1):

        #If Page Selected is Diffrent to Current Page Proceed to Get Manga list of selcted search page
        if(change['new'] != Cur_Search_Page):
          Cur_Search_Page = change['new']
          Search_List_fun(0)

      #index 2 is for Manga Chapter Image Pages    
      elif(ind==2):

        #If Page Selected is Diffrent to Current Page Proceed to Display the selected Image page
        if(change['new'] != Cur_Man_Page):
          Cur_Man_Page = change['new']
          Final_Disp(1,ind=2)

#Function to Display Widgets  
def Final_Disp(Act, ind=0):

    #Act=0 is to Display only Title and Search Widgets 
    if(Act==0):
      clear_output()
      display(VBox(Vbox_1,layout = Layout(border='1px solid black', align_items='center')))

    #Act =1 id to Display both Search List and Image Pages
    if(Act==1):

      #Initialize local variables
      Content_Box = []
      Page_Nav_Box = []
      Search_Label_Box = []
      Chap_Sel_Box = []
      Page_Sel_Box = []
      Cur_Man_Box = []

      First_Page = None
      Prev_Page = None
      Next_Page = None
      Last_Page = None
      Prev_Chap = None
      Next_Chap = None



      #index 1 for Assigning Variables/widgets for Search List Display
      if (ind==1):

        #Update local genreal page count variables with global Search list page count variables 
        Tot_Page = Tot_Search_Page
        Cur_Page = Cur_Search_Page

        #Button widgets(Disabled) to diplay Sl No, Manga Name, Latest Chapter Labels
        T_1 =  widgets.Button(description='SL. NO.', disabled=True,layout = widgets.Layout(border='solid',min_width='300px', height='30px'))
        T_2 =  widgets.Button(description='MANGA NAME', disabled=True,layout = widgets.Layout(border='solid',min_width='800px', height='30px'))
        T_3 =  widgets.Button(description='LATEST CHAPTER', disabled=True,layout = widgets.Layout(border='solid',min_width='300px', height='30px'))
        Search_Label_Box = [HBox([T_1,T_2,T_3],layout = Layout(border='1px solid black', align_items='center'))]
        
        #Get List of Search Pages for Searched manga keyword pages
        Page_List = [x for x in range(1,Tot_Search_Page+1)]
        
        #To Add Widgets of SL no, manga search List, Lastest Chapter List
        for index in range(0,len(Search_List)):

            #Call Get_Chap_list to get list of chapter and display latest chapter images if Manga button clicked
            Search_List[index][1].on_click(functools.partial(Get_Chap_List, ind=index))

            #Add SL no, manga search List, Lastest Chapter widgets to a local Hbox
            Content_Box.append(HBox(Search_List[index][0:3],layout = Layout(border='1px solid black', align_items='center')))
        
      #index 2 for Assigning Variables/widgets for Chapter Image Page Display  
      if (ind==2):
        
        #Update local general page count variables with global Manga current Chapter Image pages count variables
        Tot_Page = Tot_Man_Page
        Cur_Page = Cur_Man_Page

        #Widget to Display Current Manga Title
        Cur_Man_Box = [ widgets.Button(description=Search_List[Cur_Man][3], disabled=True,layout = widgets.Layout(border='solid',min_width='1600px', height='30px'))]

        #Dropdown Widget to select Chapters from Chapter List
        Chap_Sel_Box = widgets.Dropdown(options=Chap_Title_List,value= Chap_Title_List[Tot_Chap - Cur_Chap],description='Chapter:' ,layout = widgets.Layout(border='solid',min_width='1000px'))
        
        #Call On_Change_Chapter function when a Chapter is selected from Chapter list
        Chap_Sel_Box.observe(On_Change_Chapter)

        #get List of chapter select widget
        Chap_Sel_Box = [Chap_Sel_Box]

        #For Chapter Navigation (next, Previous) in Image Display Page
        #if number of manag chapter is 1 don't add next,previous chapter button widget 
        if(Tot_Chap == 1 ):  
          pass

        #if Current Chapter is 1 then only add Next Chapter button widget  
        elif(Cur_Chap == 1 ):
          Next_Chap = widgets.Button(description='Next Chapter', disabled=False,layout = widgets.Layout(border='solid',min_width='150px', height='30px'))

          #if Next Chap button is clicked Display 1st image of next chapter by calling Disp_Image function
          Next_Chap.on_click(functools.partial(Disp_Image,ind=Cur_Chap+1))

        #if Current Chapter is Last Chapter then only add Previous Chapter button widget  
        elif(Cur_Chap == Tot_Chap):
          Prev_Chap = widgets.Button(description='Prev Chapter', disabled=False,layout = widgets.Layout(border='solid',min_width='150px', height='30px'))

          #if Previous Chapter button is clicked Display 1st image of previous chapter by calling Disp_Image function
          Prev_Chap.on_click(functools.partial(Disp_Image,ind=Cur_Chap-1))

        #if Current Chapter is between first to Last Chapter then add both next, Previous Chapter button widget
        else:
          Prev_Chap = widgets.Button(description='Prev Chapter', disabled=False,layout = widgets.Layout(border='solid',min_width='150px', height='30px'))

          #if Next Chap button is clicked Display 1st image of next chapter by calling Disp_Image function
          Prev_Chap.on_click(functools.partial(Disp_Image,ind=Cur_Chap-1))

          Next_Chap = widgets.Button(description='Next Chapter', disabled=False,layout = widgets.Layout(border='solid',min_width='150px', height='30px'))
          #if Previous Chapter button is clicked Display 1st image of previous chapter by calling Disp_Image function
          Next_Chap.on_click(functools.partial(Disp_Image,ind=Cur_Chap+1)) 

        #Get List of Image Page nos for Selected chapter of current manga
        Page_List = [x for x in range(1,Tot_Man_Page+1)]
        
        #To Add Widget to display current Image page
        Content_Box=[widgets.HTML(value=f'<img src={Image_list[Cur_Man_Page-1]}>')]
      
      #To display Page Selection Dropdown for both Search list display and manga Image Display
      #Page_List will be for Search List or Image Display depending on Index
      Page_Sel_Box = widgets.Dropdown(options=Page_List,value= Cur_Page,description='Page:' ,layout = widgets.Layout(border='solid',min_width='400px'))
      Page_Sel_Box.observe(functools.partial(On_Change_Page,ind=ind))
      Page_Sel_Box = [HBox([Page_Sel_Box, widgets.Button(description='of '+str(Tot_Page), disabled=True,layout = widgets.Layout(border='solid',min_width='200px'))],layout = Layout(border='1px solid black', align_items='center'))]
      
      #For Page Navigation (First,Previous,Next,Last) in Image Display Page
      #if number of pages in a chapter of manga or in Search list is 1 don't add First,Previous,Next,Last page button widget
      if(Tot_Page == 1):
        pass

      #if Current Page is 1 then only add Next Page and Last Page button widget
      elif(Cur_Page == 1 ):
        Next_Page = widgets.Button(description='Next Page', disabled=False,layout = widgets.Layout(border='solid',min_width='150px', height='30px'))

        #if Next Page button is clicked Display next image of current chapter or next page of Search list by calling Nav_page function
        Next_Page.on_click(functools.partial(Nav_Page,tag=2,ind=ind))

        Last_Page = widgets.Button(description='Last Page', disabled=False,layout = widgets.Layout(border='solid',min_width='150px', height='30px'))

        #if Last Page button is clicked Display Last image of current chapter or Last page of Search list by calling Nav_page function
        Last_Page.on_click(functools.partial(Nav_Page,tag=3,ind=ind))

      #if Current Page is Last Page then only add Fisrt Page and Previous Page button widget
      elif(Cur_Page == Tot_Page):
        First_Page = widgets.Button(description='First Page', disabled=False,layout = widgets.Layout(border='solid',min_width='150px', height='30px'))

        #if First Page button is clicked Display First image of current chapter or First page of Search list by calling Nav_page function
        First_Page.on_click(functools.partial(Nav_Page,tag=0,ind=ind))

        Prev_Page = widgets.Button(description='Prev Page', disabled=False,layout = widgets.Layout(border='solid',min_width='150px', height='30px'))

        #if Previous Page button is clicked Display Previous image of current chapter or Previous page of Search list by calling Nav_page function
        Prev_Page.on_click(functools.partial(Nav_Page,tag=1,ind=ind))

      #if Current Page is between 1st page and Last Page then add (First,Previous,Next,Last) Page button widget  
      else:
        First_Page = widgets.Button(description='First Page', disabled=False,layout = widgets.Layout(border='solid',min_width='150px', height='30px'))

        #if First Page button is clicked Display First image of current chapter or First page of Search list by calling Nav_page function
        First_Page.on_click(functools.partial(Nav_Page,tag=0,ind=ind))
        
        Prev_Page = widgets.Button(description='Prev Page', disabled=False,layout = widgets.Layout(border='solid',min_width='150px', height='30px'))

        #if Previous Page button is clicked Display Previous image of current chapter or Previous page of Search list by calling Nav_page function
        Prev_Page.on_click(functools.partial(Nav_Page,tag=1,ind=ind))

        Next_Page = widgets.Button(description='Next Page', disabled=False,layout = widgets.Layout(border='solid',min_width='150px', height='30px'))

        #if Next Page button is clicked Display next image of current chapter or next page of Search list by calling Nav_page function
        Next_Page.on_click(functools.partial(Nav_Page,tag=2,ind=ind))

        Last_Page = widgets.Button(description='Last Page', disabled=False,layout = widgets.Layout(border='solid',min_width='150px', height='30px'))

        #if Last Page button is clicked Display Last image of current chapter or Last page of Search list by calling Nav_page function
        Last_Page.on_click(functools.partial(Nav_Page,tag=3,ind=ind))
      
      #Add Chapter and Page navigation widget to list
      Page_Nav_Box = [Prev_Chap,First_Page,Prev_Page,Next_Page,Last_Page,Next_Chap]

      #remove Elements for which widgets are not assigned
      Page_Nav_Box = list(filter(lambda a: a != None, Page_Nav_Box))

      #Add Chapter and Page navigation widget buttons to a horizontal box
      Page_Nav_Box = [HBox(Page_Nav_Box,layout = Layout(border='1px solid black', align_items='center'))]
      
      #Clear Output Before displaying Next Display Screen 
      clear_output()

      #Display all the Added Widgets
      display(VBox(Vbox_1+Cur_Man_Box+Chap_Sel_Box+Page_Nav_Box+Page_Sel_Box+Search_Label_Box+Content_Box+Page_Sel_Box+Page_Nav_Box,layout = Layout(border='1px solid black', align_items='center')))

if __name__ == "__main__":
  #Listen for on click of Search Buttun  
  Search_Button.on_click(Search_List_fun)

  #Display Title of App and Search box for starting the App 
  Final_Disp(0)